# Customer 360 Notebook Dashboard
AI-Driven Data Integration Quality for Multi-Source Analytics Overview

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import os
import ipywidgets as widgets
from ipywidgets import interact

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

OUTPUT_CSV_PATH = "customer_360_final.csv" 

def load_data(file_path=OUTPUT_CSV_PATH):
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            numeric_cols = ['total_spend', 'num_orders', 'total_time_spent_seconds', 'days_since_last_order']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            if 'segment' in df.columns:
                 df['segment'] = df['segment'].astype(str)
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return pd.DataFrame()
    else:
        print(f"Data file '{file_path}' not found. Run main.py first.")
        return pd.DataFrame()

df_360 = load_data()

# Customer 360 Notebook Dashboard
AI-Driven Data Integration Quality for Multi-Source Analytics Overview

In [ ]:
if not df_360.empty:
    display(HTML("<h2>Key Performance Indicators (KPIs)</h2>"))
    
    total_customers = df_360['master_customer_id'].nunique() if 'master_customer_id' in df_360.columns else 0
    total_spend = df_360['total_spend'].sum() if 'total_spend' in df_360.columns else 0
    avg_orders = df_360['num_orders'].mean() if 'num_orders' in df_360.columns and df_360['num_orders'].notna().any() else 0
    vip_count = df_360['is_vip'].sum() if 'is_vip' in df_360.columns else "N/A"

    kpi_html = f"""
    <div style='display: flex; justify-content: space-around;'>
        <div style='border: 1px solid #ccc; padding: 10px; margin: 5px; text-align: center; min-width: 200px;'>
            <h4>Total Unique Customers</h4><p style='font-size: 24px;'>{total_customers:,}</p>
        </div>
        <div style='border: 1px solid #ccc; padding: 10px; margin: 5px; text-align: center; min-width: 200px;'>
            <h4>Total Revenue</h4><p style='font-size: 24px;'>${total_spend:,.2f}</p>
        </div>
        <div style='border: 1px solid #ccc; padding: 10px; margin: 5px; text-align: center; min-width: 200px;'>
            <h4>Avg. Orders / Customer</h4><p style='font-size: 24px;'>{avg_orders:.2f}</p>
        </div>
        <div style='border: 1px solid #ccc; padding: 10px; margin: 5px; text-align: center; min-width: 200px;'>
            <h4>VIP Customers</h4><p style='font-size: 24px;'>{vip_count:,}</p>
        </div>
    </div>
    """
    display(HTML(kpi_html))
else:
    display(HTML("<p style='color:red;'>No data to display. Ensure 'customer_360_final.csv' exists.</p>"))

In [ ]:
if not df_360.empty and 'segment' in df_360.columns and df_360['segment'].nunique() > 0:
    display(HTML("<h2>Customer Segments Overview</h2>"))

    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    segment_counts = df_360['segment'].value_counts()
    sns.barplot(x=segment_counts.index, y=segment_counts.values, ax=axes[0], palette="viridis", hue=segment_counts.index, legend=False)
    axes[0].set_title("Customer Distribution by Segment")
    axes[0].set_xlabel("Segment ID")
    axes[0].set_ylabel("Number of Customers")
    for i, v in enumerate(segment_counts.values):
        axes[0].text(i, v + 0.5, str(v), color='black', ha='center')

    if 'total_spend' in df_360.columns and df_360['total_spend'].notna().any():
        segment_spend = df_360.groupby('segment')['total_spend'].mean().sort_values(ascending=False)
        sns.barplot(x=segment_spend.index, y=segment_spend.values, ax=axes[1], palette="magma", hue=segment_spend.index, legend=False)
        axes[1].set_title("Average Spend by Segment")
        axes[1].set_xlabel("Segment ID")
        axes[1].set_ylabel("Average Spend ($)")
        axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    else:
        axes[1].text(0.5, 0.5, 'Total spend data not available for segments.', 
                     horizontalalignment='center', verticalalignment='center', 
                     transform=axes[1].transAxes, fontsize=12, color='grey')
        axes[1].set_title("Average Spend by Segment")
        axes[1].set_xlabel("Segment ID")
        axes[1].set_ylabel("Average Spend ($)")

    plt.tight_layout()
    plt.show()
elif not df_360.empty:
    display(HTML("<p>Segmentation data not available or not diverse for display.</p>"))

In [ ]:
if not df_360.empty:
    display(HTML("<h2>Customer Data Table (Sample)</h2>"))
    display(df_360.head(10))


In [ ]:
if not df_360.empty and 'email' in df_360.columns and df_360['email'].notna().any():
    display(HTML("<h3>Individual Customer Profile (Interactive)</h3>"))
    email_options = [""] + sorted(df_360['email'].astype(str).dropna().unique().tolist())
    
    @interact(email=widgets.Dropdown(options=email_options, description='Select Email:', layout={'width': '400px'}))
    def show_customer_profile(email):
        if email:
            profile = df_360[df_360['email'] == email]
            if not profile.empty:
                display(HTML(profile.T.to_html()))
            else:
                print("Customer not found.")
        else:
            print("Select an email to view profile.")